In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

resnet_model = ResNet50(input_shape=(224, 224, 3), include_top = False) # 출력층을 제외하고 모델을 가져온다.

In [ ]:
resnet_model.trainable = True
model = Sequential([
    resnet_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 100352)            0         
                                                                 
 dense_2 (Dense)             (None, 128)               12845184  
                                                                 
 dense_3 (Dense)             (None, 3)                 387       
                                                                 
Total params: 36433283 (138.98 MB)
Trainable params: 36380163 (138.78 MB)
Non-trainable params: 53120 (207.50 KB)
_________________________________________________________________


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# train 데이터 Image augmentation
train_gen = ImageDataGenerator(rotation_range=20, # 회전 각도
                               width_shift_range=0.2, # 해당 속성의 값을 정수로 주면 화소의 범위를, 실수를 주면 비율을 단위로 한다.
                               height_shift_range=0.2,
                               horizontal_flip=True # 좌우반전
                              )
train_data = train_gen.flow_from_directory('./glaucoma/train', target_size=(224, 224), class_mode='sparse')

# test 데이터 Image augmentation
test_gen = ImageDataGenerator()
test_data = test_gen.flow_from_directory('./glaucoma/test', target_size=(224, 224), class_mode='sparse')

Found 1394 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train_data, validation_data=test_data, epochs=20)

Epoch 1/20
44/44 [==============================] - 235s 5s/step - loss: 3.3803 - accuracy: 0.6628 - val_loss: 3341002.7500 - val_accuracy: 0.5200
Epoch 2/20
44/44 [==============================] - 215s 5s/step - loss: 0.7258 - accuracy: 0.7138 - val_loss: 638.2086 - val_accuracy: 0.5200
Epoch 3/20
44/44 [==============================] - 216s 5s/step - loss: 0.6159 - accuracy: 0.7346 - val_loss: 1.2483 - val_accuracy: 0.7267
Epoch 4/20
44/44 [==============================] - 216s 5s/step - loss: 0.5291 - accuracy: 0.7489 - val_loss: 2.1814 - val_accuracy: 0.5933
Epoch 5/20
44/44 [==============================] - 217s 5s/step - loss: 0.5787 - accuracy: 0.7611 - val_loss: 0.9373 - val_accuracy: 0.6800
Epoch 6/20
44/44 [==============================] - 221s 5s/step - loss: 0.5539 - accuracy: 0.7704 - val_loss: 2.7335 - val_accuracy: 0.5400
Epoch 7/20
44/44 [==============================] - 217s 5s/step - loss: 0.5097 - accuracy: 0.7798 - val_loss: 1.1003 - val_accuracy: 0.6733
Epoch

In [ ]:
model.save('resnet_glaucoma.keras')

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img = image.load_img("ad_glaucoma_sample.png", target_size=(224, 224))
x = image.img_to_array(img).reshape(-1, 224, 224, 3)
pred = model.predict(x) # 확률값을 가진다.
print(np.argmax(pred, axis=1)) # 0:advanced_glaucoma, 1:early_glaucoma, 2:normal_control

1/1 [==============================] - 1s 914ms/step
[0]
